اول از همه لایبرری های مورد نیاز رو ایمپورت کنیم به پروژه

In [48]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

لیست ژانر ها رو داریم و در نظر می گیریم مثلا هر ژانری ماکسیمم 1000 تا صفحه داره و بنابراین به ازای هر ژانر از صفحه 0 تا 1000 توی دو تا حلقه تو در تو ریکوئست میزنیم و دیتای اون ژانر رو می گیریم.

حواستون باشه که اجرای تیکه کد پایین به اضای 1000 صفحه از هر ژانر کلی زمان احتیاج داره.

به ازای هر ریکوئستی که زده میشه، لیست فیلم ها تو قالب HTML از توی ریکوئست استخراج میشه و و به یه لیست فلت که شامل همه فیلم ها تو همه ژانر ها هست اضافه میشه.

In [ ]:
genres = ["action","adventure","animation","biography","comedy","crime","drama","family","fantasy",
          "film_noir","history","horror","music","musical","mystery","romance","sci_fi","sport",
          "thriller","war","western"]

movies_div_list = []

for genre in genres:
    i=0
    while(True):
        req = requests.get('https://www.imdb.com/search/title/?genres='+str(genre)+'&start='+str((i*50)+1))
        if(req.status_code == 200 and i<1000):
            soup = bs(req.content)
            temp_movies_list = soup.find_all('div', attrs={'class':'lister-item mode-advanced'})
            for movie in temp_movies_list:
                movies_div_list.append(movie)
        else:
            break
        i+=1


تو تیکه کد پایین می خواییم لیست رو پیمایش کنیم و به ازای هر آیتم که شامل اطلاعات فیلم در قالب HTML هست، اطلاعات فیلم رو استخراج کنیم و در قالب یک لیست تو در تو در بیاریم که بعدش بتونیم باهاش یه جدول درست کنیم و تو فایل CSV ذخیره کنیم.

In [ ]:
list_of_movies_details = []
for movie_html in movies_div_list:
    list_of_fields = []
    # for item's title
    item_title = movie_html.select('h3.lister-item-header a')[0].text
    list_of_fields.append(item_title)

    #for item's year
    item_year = movie_html.select('span.lister-item-year')[0].text
    list_of_fields.append(item_year)

    #for item's certificate
    item_certificate = ''
    if(len(movie_html.select('span.certificate'))>0):
        item_certificate = movie_html.select('span.certificate')[0].text
    list_of_fields.append(item_certificate)

    #for item's length
    item_length = 0
    if(len(movie_html.select('span.runtime'))>0):
        item_length = movie_html.select('span.runtime')[0].text
    list_of_fields.append(item_length)

    #for item's genres
    item_genres = movie_html.select('span.genre')[0].text
    list_of_fields.append(item_genres)

    #for item's score
    item_score = ''
    if(len(movie_html.select('div.inline-block strong'))>0):
        item_score = movie_html.select('div.inline-block strong')[0].text
    list_of_fields.append(item_score)

    #for item's metascore
    item_metascore = ''
    if(len(movie_html.select('span.metascore'))>0):
        item_metascore = movie_html.select('span.metascore')[0].text
    list_of_fields.append(item_metascore)

    #for item's summary
    item_summary = movie_html.select('p.text-muted')[1].text
    list_of_fields.append(item_summary)

    #for item's director and stars
    director_and_stars_html = movie_html.select('div.lister-item-content p')[2]
    item_stars_list = [a.text for a in director_and_stars_html.select('a')]
    item_director =''
    if(len(director_and_stars_html.select('span.ghost'))>0):
        item_director = item_stars_list.pop(0)
    list_of_fields.append(item_director)
    list_of_fields.append(item_stars_list)
        
    #for item's votes
    item_votes = 0
    if(len(movie_html.select('p.sort-num_votes-visible span'))>1):
        item_votes = movie_html.select('p.sort-num_votes-visible span')[1]['data-value']
    list_of_fields.append(item_votes)

    list_of_movies_details.append(list_of_fields)


تو تیکه کد پایین باید جدول به دست اومده از اطلاعات فیلم ها رو ذخیره می کنیم توی یه فایل سی اس وی

In [ ]:
df = pd.DataFrame(list_of_movies_details, columns=['title', 'year', 'certificate', 'length','genres', 'score', 'metascore','summary',
                                                   'director', 'stars_list', 'votes'])
df.to_csv('imdb_movies.csv',index=False,sep=';')